In [ ]:
from rdkit.Chem import Descriptors, AllChem as Chem, DataStructs
from rdkit.Chem import PandasTools
import numpy as np
import joblib
import pickle
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing

In [ ]:
def calc_descriptors(rdmol):
    try:
        logp = Descriptors.MolLogP(rdmol)
        mwt = Descriptors.MolWt(rdmol)
        rtb = Descriptors.NumRotatableBonds(rdmol)
        hbd = Descriptors.NumHDonors(rdmol)
        hba = Descriptors.NumHAcceptors(rdmol)
        tpsa = Descriptors.TPSA(rdmol)
        return [logp, mwt, rtb, hbd, hba, tpsa]
    except:
        return [999, 999, 999, 999, 999, 999, 999]
    
def calc_fingerprint(rdmol, N_BITS=1024):
        fp = Chem.GetMorganFingerprintAsBitVect(
            rdmol, radius=2, nBits=N_BITS, useFeatures=False
        )
        np_fp = np.zeros(0)
        ecfp = DataStructs.ConvertToNumpyArray(fp, np_fp)
        return np_fp

def pred_category(p0, p1, significance):
    if (p0 >= significance) & (p1 >= significance):
        return "both"
    if (p0 >= significance) & (p1 < significance):
        return "inactive"
    if (p0 < significance) & (p1 >= significance):
        return "active"
    else:
        return "empty"

In [ ]:
df = pd.read_csv('herg_compound_from_chembl_processed_largestFrag_delDupl.csv')
df = df[['activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value', 'smiles']]

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df, 'smiles', 'mol')
df['mol'] = df['mol'].fillna(0)
df = df[df.mol != 0]
df = df[df.standard_relation == '=']

In [ ]:
def convert_IC50_pIC50(value, unit='nM'):
    if unit == 'nM':
        p_value = -np.log10(value/1e9)
    elif unit == 'uM':
        p_value = -np.log10(value/1e6)
    return p_value
        
df['pIC50'] = [convert_IC50_pIC50(x) for x in df.standard_value.values]

In [ ]:
df.pIC50.plot(kind='hist')

In [ ]:
features = [np.append(calc_descriptors(mol), calc_fingerprint(mol)) for mol in df.mol]

In [ ]:
import os
import pickle

scaler = preprocessing.StandardScaler().fit(features)

if not os.path.exists('models'):
    os.makedirs('models')
pickle.dump(scaler, open('models/scaler.pkl','wb'))

X= scaler.transform(features)
y = df['pIC50'].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from nonconformist.base import RegressorAdapter
from nonconformist.icp import IcpRegressor
from nonconformist.nc import MarginErrFunc
from nonconformist.nc import RegressorNc, RegressorNormalizer
from nonconformist.nc import AbsErrorErrFunc, SignErrorErrFunc

from nonconformist.evaluation import cross_val_score
from nonconformist.evaluation import RegIcpCvHelper
from nonconformist.evaluation import reg_mean_errors, reg_median_size

icp = IcpRegressor(
    RegressorNc(
        RegressorAdapter(RandomForestRegressor(n_estimators=100)), AbsErrorErrFunc()
    )
)
# icp_cv = RegIcpCvHelper(icp)

# scores = cross_val_score(
#     icp_cv,
#     X,
#     y,
#     iterations=5,
#     folds=5,
#     scoring_funcs=[reg_mean_errors, reg_median_size],
#     significance_levels=[0.05, 0.1, 0.2],
# )

In [ ]:
# scores = scores.drop(["fold", "iter"], axis=1)
# print(scores.groupby(["significance"]).mean())

In [ ]:
n_instances = y.size
idx = np.random.permutation(n_instances)

train_idx = idx[: 4*int(n_instances / 8)]
cal_idx = idx[4*int(n_instances / 8) : 7*int(n_instances / 8) ]
test_idx = idx[7*int(n_instances / 6) :]

icp.fit(X[train_idx, :], y[train_idx])
icp.calibrate(X[cal_idx, :], y[cal_idx])

if not os.path.exists('models'):
    os.makedirs('models')
pickle.dump(icp, open('models/icp.pkl','wb'))

In [ ]:
# icp.cal_scores.values()

In [ ]:
#Load models and predict molecules (smiles)

scaler = pickle.load(open('models/scaler.pkl', 'rb'))
icp = pickle.load(open('models/icp.pkl', 'rb'))

In [ ]:
df_results = pd.DataFrame(icp.predict(X[test_idx, :], significance=0.1), columns=['Minmum','Maximum'])
df_results['Real'] = y[test_idx].reshape(-1)

In [ ]:
smiles = '''C1=CC=C(C=C1)C=O
CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@@H]1[C@H]([C@H]([C@](O1)(C#N)C2=CC=C3N2N=CN=C3N)O)O)OC4=CC=CC=C4
CCN(CC)CCCC(C)NC1=C2C=CC(=CC2=NC=C1)Cl'''
smiles = smiles.split()

In [ ]:
mols = [Chem.MolFromSmiles(s) for s in smiles]

In [ ]:
features = [np.append(calc_descriptors(mol), calc_fingerprint(mol)) for mol in pf_mols]
X = scaler.transform(features)

In [ ]:
significance = 0.1
prediction = pd.DataFrame(icp.predict(X, significance=significance), columns=['Minmum','Maximum'])
prediction['Significance'] = '{}%' .format(100-int(significance*100))

In [ ]:
prediction